In [1]:
# Table 1: Subjective and Objective Literacy Score Summaries 
import pandas as pd

# Load data
df = pd.read_csv('d:/data/python/NFCS2021.csv')

# Clean subjective literacy (M4)
df['M4'] = pd.to_numeric(df['M4'], errors='coerce')
df['M4'] = df['M4'].where(~df['M4'].isin([97, 98, 99]))

# Define correct answers for M6–M10
answers = {'M6': 1, 'M7': 3, 'M8': 2, 'M9': 1, 'M10': 2}

# Score objective literacy
df['Objective'] = sum((pd.to_numeric(df[q], errors='coerce') == a).astype(int) for q, a in answers.items())

# Compute and print statistics
for col in ['M4', 'Objective']:
    print(f"{col} — Mean: {round(df[col].mean(), 2)}, Median: {round(df[col].median(), 2)}, Std Dev: {round(df[col].std(), 2)}")	




M4 — Mean: 5.07, Median: 5.0, Std Dev: 1.34
Objective — Mean: 2.66, Median: 3.0, Std Dev: 1.53


In [1]:
# Reproduce Table 1 using OpenAI API 
import pandas as pd
import openai
from openai import OpenAI

# Load and clean the data
df = pd.read_csv('d:/data/python/NFCS2021.csv')

# Clean subjective literacy (M4)
df['M4'] = pd.to_numeric(df['M4'], errors='coerce')
df['M4'] = df['M4'].where(~df['M4'].isin([97, 98, 99]))

# Score objective literacy (M6–M10)
answers = {'M6': 1, 'M7': 3, 'M8': 2, 'M9': 1, 'M10': 2}
df['Objective'] = sum((pd.to_numeric(df[q], errors='coerce') == a).astype(int) for q, a in answers.items())

# Calculate summary statistics
summary_stats = {
    "Subjective Literacy (M4)": {
        "Mean": round(df['M4'].mean(), 2),
        "Median": round(df['M4'].median(), 2),
        "Std Dev": round(df['M4'].std(), 2)
    },
    "Objective Literacy (M6–M10)": {
        "Mean": round(df['Objective'].mean(), 2),
        "Median": round(df['Objective'].median(), 2),
        "Std Dev": round(df['Objective'].std(), 2)
    }
}

# Create a prompt for explanation
prompt = (
    "Here are financial literacy statistics from a national survey:\n\n"
    f"{summary_stats}\n\n"
    "Explain these findings in simple terms for a policy audience interested in consumer financial capability."
)

# Initialize OpenAI client (requires openai>=1.0.0)
client = OpenAI(api_key="")   # add your open AI API key

# Call the model using client.chat.completions.create
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful data analyst."},
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(response.choices[0].message.content)


These statistics present findings from a national survey on financial literacy, covering two distinct measurements: 'Subjective Literacy' and 'Objective Literacy'.

'Subjective Literacy (M4)' measures how individuals assess their own competence in financial matters. On average, individuals rate their financial literacy at 5.07 (on a scale likely ranging from 1 to 7, with higher scores indicating higher literacy). In simpler terms, people generally see themselves as slightly above average when it comes to financial know-how.

However, this self-assessed competency comes with some variability. The standard deviation value of 1.34 gives us an idea of how much people’s views of their own literacy vary around this average. Some people rate themselves quite high, while others are more modest in their self-assessment.

The 'Objective Literacy (M6–M10)' probably tests a person's concrete financial knowledge with specific questions. Here, we find less optimism. The average score is only 2.66, a

In [11]:
# Table 2: ANOVA Results: Objective Literacy Scores by Income Group

import pandas as pd
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols

# === Step 1: Load data ===
file_path = r'd:\data\python\nfcs2021.csv'  # Update as needed
df = pd.read_csv(file_path)

# === Step 2: Define correct answers and compute objective score ===
correct_answers = {'M6': 1, 'M7': 3, 'M8': 2, 'M9': 1, 'M10': 2}
df['objective_score'] = df[list(correct_answers)].eq(pd.Series(correct_answers)).sum(axis=1)

# === Step 3: Income group mapping ===
income_map = {
    1: '<$15k',
    2: '$15k–$25k',
    3: '$25k–$35k',
    4: '$35k–$50k',
    5: '$50k–$75k',
    6: '$75k–$100k',
    7: '$100k–$150k',
    8: '$150k–$200k',
    9: '$200k–$300k',
    10: '$300k+'
}
df['income_group'] = df['A5_2015'].map(income_map)

# === Step 4: Clean and filter ===
df_clean = df[['income_group', 'objective_score']].dropna()
df_clean = df_clean[df_clean['objective_score'].between(0, 7)]

# === Step 5: Summary stats ===
summary = df_clean.groupby('income_group')['objective_score'].agg(
    Mean='mean',
    Std='std',
    Count='count'
).round(2).reset_index()

print("\n--- Summary: Objective Financial Literacy by Income Group ---")
print(summary.to_string(index=False))

# === Step 6: ANOVA ===
anova_model = ols('objective_score ~ C(income_group)', data=df_clean).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)

print("\n--- ANOVA Results ---")
print(anova_table)



--- Summary: Objective Financial Literacy by Income Group ---
income_group  Mean  Std  Count
 $100k–$150k  3.36 1.45   2969
   $15k–$25k  2.00 1.41   4825
   $25k–$35k  1.93 1.38   1897
   $35k–$50k  2.59 1.45   7065
   $50k–$75k  2.76 1.46   2943
  $75k–$100k  3.22 1.45   6682
       <$15k  1.42 1.26    737

--- ANOVA Results ---
                       sum_sq       df           F  PR(>F)
C(income_group)   7834.405930      6.0  635.983115     0.0
Residual         55661.482742  27111.0         NaN     NaN


In [1]:
import pandas as pd
import statsmodels.formula.api as smf

# Load data from your specified path
df = pd.read_csv(r'd:\data\python\NFCS2021.csv')  # Use raw string (r'') for Windows paths

# Compute objective literacy score from M6–M10 (optional)
answers = {'M6': 1, 'M7': 3, 'M8': 2, 'M9': 1, 'M10': 2}
df['Objective'] = sum((pd.to_numeric(df[q], errors='coerce') == a).astype(int) for q, a in answers.items())

# Prepare dependent variable: subjective literacy (M4)
df['subjective'] = pd.to_numeric(df['M4'], errors='coerce')

# Independent variables
df['female'] = df['A4A_new_w'].apply(lambda x: 1 if x == 2 else 0)
df['age_group'] = pd.to_numeric(df['A3Ar_w'], errors='coerce')
df['income'] = df['A5_2015'].astype('category')        # Categorical income
df['married'] = df['A6'].apply(lambda x: 1 if x == 1 else 0)
df['children'] = pd.to_numeric(df['A7'], errors='coerce')
df['educ'] = df['A8_2021'].astype('category')          # Categorical education

# Drop missing data
df_reg = df[['subjective', 'female', 'age_group', 'income', 'married', 'children', 'educ']].dropna()

# Run regression: subjective literacy as dependent variable
model = smf.ols('subjective ~ female + age_group + C(income) + married + children + C(educ)', data=df_reg).fit()

# Print regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             subjective   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     15.72
Date:                Sat, 07 Jun 2025   Prob (F-statistic):           7.59e-52
Time:                        16:09:25   Log-Likelihood:            -1.1049e+05
No. Observations:               27118   AIC:                         2.210e+05
Df Residuals:                   27098   BIC:                         2.212e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         10.7506      0.629     17.

In [2]:
import pandas as pd
import numpy as np
import os

# Set folder path where the NFCS CSV files are located
folder_path = 'd:/data/python/'  # Adjust this to your actual path

# List of survey years
waves = ['2009', '2012', '2015', '2018', '2021']
results = []

# Loop through each year and compute statistics
for wave in waves:
    file_path = os.path.join(folder_path, f'NFCS{wave}.csv')
    
    try:
        df = pd.read_csv(file_path)
        
        # Clean subjective financial literacy (M4)
        m4 = pd.to_numeric(df['M4'], errors='coerce')
        m4 = m4.where(~m4.isin([97, 98, 99]))  # Remove invalid responses
        
        # Calculate mean and standard deviation
        mean = round(m4.mean(), 2)
        std = round(m4.std(), 2)
        
        results.append({'Year': wave, 'Mean': mean, 'Standard Deviation': std})
    
    except Exception as e:
        results.append({'Year': wave, 'Mean': 'Error', 'Standard Deviation': str(e)})

# Convert to DataFrame and display
summary_df = pd.DataFrame(results)
print(summary_df)


   Year  Mean  Standard Deviation
0  2009  4.99                1.29
1  2012  5.17                1.28
2  2015  5.26                1.20
3  2018  5.13                1.35
4  2021  5.07                1.34


In [13]:
import pandas as pd
import numpy as np
import os

# Set your data folder path
folder_path = 'd:/data/python/'  # Change this if needed
waves = ['2009', '2012', '2015', '2018', '2021']

# Correct answers for M6–M10
correct_answers = {'M6': 1, 'M7': 3, 'M8': 2, 'M9': 1, 'M10': 2}

results = []

for wave in waves:
    file_path = os.path.join(folder_path, f'NFCS{wave}.csv')
    
    try:
        df = pd.read_csv(file_path)

        # Score objective literacy
        score_df = pd.DataFrame({
            q: (pd.to_numeric(df[q], errors='coerce') == ans).astype(int)
            for q, ans in correct_answers.items()
        })

        # Total correct per respondent
        df['Total_Correct'] = score_df.sum(axis=1)

        # Drop missing
        valid_scores = df['Total_Correct'].dropna()

        # Calculate stats
        mean = round(valid_scores.mean(), 2)
        std = round(valid_scores.std(), 2)
        pct_ge_4 = round((valid_scores >= 4).mean() * 100, 1)

        results.append({
            'Year': wave,
            'Mean Correct (out of 5)': mean,
            'Standard Deviation': std,
            '% Answering ≥ 4 Correct': pct_ge_4
        })

    except Exception as e:
        results.append({
            'Year': wave,
            'Mean Correct (out of 5)': 'Error',
            'Standard Deviation': 'Error',
            '% Answering ≥ 4 Correct': str(e)
        })

summary_df = pd.DataFrame(results)
print(summary_df)


   Year  Mean Correct (out of 5)  Standard Deviation  % Answering ≥ 4 Correct
0  2009                     3.12                1.42                     46.2
1  2012                     2.99                1.46                     42.1
2  2015                     2.95                1.47                     40.6
3  2018                     2.80                1.50                     36.8
4  2021                     2.66                1.53                     34.0
